<a href="https://colab.research.google.com/github/malekeechaker/ARCH_GARCH_TimeSerie_models/blob/main/3rd-week/SetUp-Dataset/injection_huggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/chYassine/WebAttack-CVSSMetrics/data/data-00000-of-00001.parquet")

In [ ]:
df.head(10)

,_raw,Type,CVSS,Score
0,91.251.15.250 - - [22/Jan/2019:18:18:42 +0330]...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
1,5.121.127.208 - - [23/Jan/2019:23:06:04 +0330]...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
2,66.249.66.194 - - [24/Jan/2019:18:12:21 +0330]...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
3,5.117.227.193 - - [24/Jan/2019:11:56:42 +0330]...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
4,5.212.17.137 - - [26/Jan/2019:17:42:43 +0330] ...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
5,5.101.40.234 - - [23/Jan/2019:06:58:11 +0330] ...,SSTI,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,1.0
6,192.168.202.110 - - [16/Mar/2012:19:38:49 +033...,Sql Injection,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:L/VI:L/VA...,5.8
7,79.127.104.91 - - [26/Jan/2019:11:00:25 +0330]...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
8,204.18.143.40 - - [24/Jan/2019:04:12:39 +0330]...,None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0
9,"5.123.90.28 - - [23/Jan/2019:22:17:34 +0330] ""...",None,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0


# Exploring web attack types in the dataset

In [ ]:
df.loc[df["Type"].isnull(), "Type"] = "Normal"


In [ ]:
df["Type"].value_counts()

Type
Normal           10000
LFI               3088
SSTI              2105
Sql Injection     1706
XSS                648
SSRF               576
file upload        437
CSRF               282
Name: count, dtype: int64

LFI (Local File Inclusion)
SSTI (Server-Side Template Injection)
SQL Injection (SQLi)
XSS (Cross-Site Scripting) are the only injection attacks

In [ ]:
filtered_df = df[df["Type"].isin(["Normal","SSTI","Sql Injection","XSS","LFI"])]


In [ ]:
filtered_df.isnull().sum()

_raw     0
Type     0
CVSS     0
Score    0
dtype: int64

In [ ]:
filtered_df['_raw'][18755]

'10.128.0.205 - - [24/Mar/2018:21:53:02 +0330] "GET //surgeweb?username_ex="><scri<script>alert(42);</script><input type="hidden HTTP/1.0" 404 151 "-" "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)" "-"'

In [ ]:
filtered_df['Payload'] = filtered_df['_raw'].str.extract(r'(GET|POST|HEAD) (.*) HTTP', expand=False)[1]



<ipython-input-155-229db945b7fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Payload'] = filtered_df['_raw'].str.extract(r'(GET|POST|HEAD) (.*) HTTP', expand=False)[1]


In [ ]:
filtered_df.isnull().sum()

_raw       0
Type       0
CVSS       0
Score      0
Payload    0
dtype: int64

In [ ]:
filtered_df = filtered_df.rename(columns={'Type': 'label'})


In [ ]:
filtered_df

,_raw,label,CVSS,Score,Payload
0,91.251.15.250 - - [22/Jan/2019:18:18:42 +0330]...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/apple-touch-icon-precomposed.png
1,5.121.127.208 - - [23/Jan/2019:23:06:04 +0330]...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/image/3/productType/120x90
2,66.249.66.194 - - [24/Jan/2019:18:12:21 +0330]...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,"/m/filter/b256,p44"
3,5.117.227.193 - - [24/Jan/2019:11:56:42 +0330]...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/image/36/productType/120x90
4,5.212.17.137 - - [26/Jan/2019:17:42:43 +0330] ...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/image/58596/productModel/200x200
...,...,...,...,...,...
18837,83.122.142.96 - - [22/Jan/2019:13:46:22 +0330]...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/m/filter/b878?page=1
18838,10.128.0.238 - - [23/Mar/2018:19:01:06 +0330] ...,SSTI,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:L/VI:L/VA...,5.5,/api/set/1/2?value=90000000
18839,94.24.96.115 - - [23/Jan/2019:02:34:06 +0330] ...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/settings/logo
18840,83.120.32.165 - - [26/Jan/2019:10:10:38 +0330]...,Normal,CVSS:4.0/AV:N/AC:L/AT:N/PR:N/UI:N/VC:N/VI:N/VA...,0.0,/image/11448/productModel/200x200


In [165]:
filtered_df.drop(columns=["_raw","CVSS","Score"],inplace=True)

In [168]:
filtered_df['Blabel'] = filtered_df['label'].map(lambda x: 0 if x == 'Normal' else 1)


In [170]:
filtered_df.drop(columns=["label"],inplace=True)
filtered_df = filtered_df.rename(columns={'Blabel': 'label'})


In [173]:
filtered_df.shape

(17547, 2)

In [172]:
filtered_df.to_csv("injection_data.csv", index=False)